In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample


/Users/swapnilgharat/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/swapnilgharat/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


# IMPORTING DATA SET AND PRE-PROCESSING

In [2]:
dataset=pd.read_csv('/Users/swapnilgharat/Desktop/520/FINAL PROJECT/Train.csv')
dataset.shape


(2500, 68)

In [3]:
dataset.y.value_counts()

-1    1891
 1     609
Name: y, dtype: int64

In [4]:
dataset= dataset.drop('Row', axis =1)

In [5]:
dataset.shape

(2500, 67)

# SINCE THERE IS A UNBALANCE BETWEEN THE CLASSES WE USE UPSAMPLING TO MAKE THE NUMBER OF INSTANCES IN EACH CLASSES EQUAL


STEPS TO CONVERT CATEGORICAL DATA TO NUMERICAL DATA
STEP1: SEPERATE AND DROP CATEGORICAL DATA




In [6]:

CAT=list(dataset.select_dtypes(include=['object']))
dpX=dataset.drop(CAT,axis =1).astype('float64')
dpX
list (CAT)


['x5', 'x13', 'x64', 'x65']

In [7]:
dpX.shape 

(2500, 63)

In [8]:
#CREATE DUMMY COLUMNS FOR CATEGORICAL DATA
d1=pd.get_dummies(dataset['x5'])
d2=pd.get_dummies(dataset['x13'])
d3=pd.get_dummies(dataset['x64'])
d4=pd.get_dummies(dataset['x65'])

In [9]:
list(d3)

['Ma', 'Mk', 'Mm', 'Mp']

In [10]:
Xnew=pd.concat([dpX,d1,d2,d3,d4],axis=1).astype('float')

In [11]:
Xnew.shape

(2500, 79)

In [12]:
import numpy as np
Xnew.fillna(np.mean(Xnew), inplace = True)
Xnew.shape

(2500, 79)

In [13]:
y = Xnew['y']


In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Xnew, y, test_size=0.2, random_state=42)

# PROCESSING TRAIN DATA(UPSAMPLING)

In [15]:
X_train.y.value_counts()

-1.0    1516
 1.0     484
Name: y, dtype: int64

In [16]:
#upsampling of the dataset
from sklearn.utils import resample
major = X_train[X_train['y']==-1]
minor = X_train[X_train['y']==1]
upsampled = resample(minor,replace=True,n_samples=1516,random_state=123)
newupsampled = pd.concat([major,upsampled])

In [17]:
newupsampled.y.value_counts()

 1.0    1516
-1.0    1516
Name: y, dtype: int64

In [18]:
ynew_train = newupsampled['y']
X = newupsampled.drop('y', axis =1)

In [19]:
X.shape

(3032, 78)

In [20]:
ynew_train.shape

(3032,)

In [21]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
XStdtrain = scaler.fit(X).transform(X)

In [22]:
XStdtrain.shape

(3032, 78)

# TEST DATA

In [23]:
X_test.shape

(500, 79)

In [24]:
ynew_test = X_test['y']
Xtest =X_test.drop('y', axis =1)

In [25]:
Xtest.shape

(500, 78)

In [26]:
XStdtest = scaler.transform(Xtest)

In [27]:
XStdtest.shape

(500, 78)

In [28]:
ynew_test.shape

(500,)

# TRIAL 1: USING SVM

In [29]:
from sklearn import svm
from sklearn.svm import SVC
modelsvm = SVC()
modelsvm.fit(XStdtrain,ynew_train)
ypred = modelsvm.predict(XStdtest)
print ("Training Accuracy")
print (metrics.accuracy_score(ynew_test, ypred)*100, "%")
print (metrics.classification_report(ynew_test, ypred))

Training Accuracy
72.0 %
             precision    recall  f1-score   support

       -1.0       0.94      0.67      0.78       375
        1.0       0.47      0.88      0.61       125

avg / total       0.82      0.72      0.74       500



In [30]:
tuned_parameters = [{'kernel': ['rbf',], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000],'C': [1, 10, 100, 1000]}]
modsvm2 = GridSearchCV(modelsvm, cv = 5 ,refit = 'true',param_grid = tuned_parameters)
modsvm2.fit(XStdtrain,ynew_train)
print(modsvm2.best_params_)

{'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}


In [31]:
modsvm3 = SVC(kernel = 'rbf', C = 1000,gamma=0.001, probability = True,class_weight = 'balanced')

In [32]:
XStdtest.shape

(500, 78)

In [33]:
ynew_test.shape

(500,)

In [34]:
modsvm3.fit(XStdtrain,ynew_train)
yprednew =modsvm3.predict(XStdtest)
from sklearn.metrics import accuracy_score
print(accuracy_score(ynew_test, yprednew))


0.786


In [35]:
from sklearn.metrics import confusion_matrix
confusion_matrix(ynew_test,yprednew)

array([[291,  84],
       [ 23, 102]])

# ACCURACY: 78.6% BALANCED ERROR RATE: 0.204

# TRIAL 2: USING DECISION TREE

In [36]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import model_selection
from sklearn.model_selection import ShuffleSplit
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import confusion_matrix
import itertools
import sklearn as skl_lm

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import ShuffleSplit
from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

In [37]:
modtree = DecisionTreeClassifier()
modtree.fit(XStdtrain,ynew_train)
ypredDT = modtree.predict(XStdtest)
print ("Training Accuracy")
print (metrics.accuracy_score(ynew_test, ypredDT)*100, "%")
print (metrics.classification_report(ynew_test, ypredDT))

Training Accuracy
76.2 %
             precision    recall  f1-score   support

       -1.0       0.83      0.86      0.84       375
        1.0       0.53      0.47      0.50       125

avg / total       0.75      0.76      0.76       500



In [38]:
tuneparameters = [{'criterion': ['entropy','gini'], 'max_depth': [1,2,3,4,5,6,7,8,9,10],'min_samples_split': [2,3,4,5,6,7,8,9,10]}]
modtree2 = GridSearchCV(modtree, cv = 8 ,refit = 'true',param_grid = tuneparameters)
modtree2.fit(XStdtrain,ynew_train)

GridSearchCV(cv=8, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'criterion': ['entropy', 'gini'], 'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10]}],
       pre_dispatch='2*n_jobs', refit='true', scoring=None, verbose=0)

In [39]:
print(modtree2.best_params_)

{'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 4}


In [72]:
modelregtree_3= DecisionTreeClassifier(criterion ='gini',max_depth = 10,min_samples_split=4)
modelregtree_3.fit(XStdtrain,ynew_train)
ypredreg = modelregtree_3.predict(XStdtest)
from sklearn.metrics import accuracy_score
print(accuracy_score(ynew_test, ypredreg))


0.752


In [73]:
from sklearn.metrics import confusion_matrix
confusion_matrix(ynew_test,ypredreg)

array([[295,  80],
       [ 44,  81]])

# ACCURACY:75.4% BALANCED ERROR RATE:0.28


# TRIAL 3:USING RANDOM FOREST

In [42]:
from sklearn.ensemble import RandomForestClassifier
seedStart = 2357
modelnowRF=RandomForestClassifier(random_state=seedStart)
modelnowRF.fit(XStdtrain,ynew_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=2357, verbose=0,
            warm_start=False)

In [43]:
ynew_test.shape

(500,)

In [44]:
ypredRF = modelnowRF.predict(XStdtest)
print ("Training Accuracy")
print (metrics.accuracy_score(ynew_test, ypredRF)*100, "%")
print (metrics.classification_report(ynew_test, ypredRF))
from sklearn.metrics import confusion_matrix
confusion_matrix(ynew_test,ypredRF)

Training Accuracy
79.4 %
             precision    recall  f1-score   support

       -1.0       0.83      0.91      0.87       375
        1.0       0.62      0.44      0.52       125

avg / total       0.78      0.79      0.78       500



array([[342,  33],
       [ 70,  55]])

In [45]:
from sklearn.grid_search import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_jobs=-1,max_features= 'sqrt' ,n_estimators=50, oob_score = True)
param_grid = {'n_estimators': [200,700], 'max_features': ['auto', 'sqrt', 'log2']}

CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 7)
CV_rfc.fit(XStdtrain,ynew_train)




GridSearchCV(cv=7, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
            oob_score=True, random_state=None, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [200, 700], 'max_features': ['auto', 'sqrt', 'log2']},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [46]:
print(CV_rfc.best_params_)


{'max_features': 'sqrt', 'n_estimators': 700}


In [69]:
rfcT = RandomForestClassifier(n_jobs=-1,max_features= 'sqrt' ,n_estimators=700, oob_score = True)
rfcT.fit(XStdtrain,ynew_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=700, n_jobs=-1,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [70]:
XStdtest.shape

(500, 78)

In [71]:
ypredrfct = rfcT.predict(XStdtest)
print ("Training Accuracy")
print (metrics.accuracy_score(ynew_test, ypredrfct)*100, "%")
print (metrics.classification_report(ynew_test, ypredrfct))
from sklearn.metrics import confusion_matrix
confusion_matrix(ynew_test,ypredrfct)

Training Accuracy
83.0 %
             precision    recall  f1-score   support

       -1.0       0.87      0.91      0.89       375
        1.0       0.69      0.58      0.63       125

avg / total       0.82      0.83      0.82       500



array([[343,  32],
       [ 53,  72]])

# Training Accuracy= 83.4 %



# Balanced error rate = 0.504/2 = 0.252


# BUILDING FINAL MODEL ON RANDOM FOREST

In [50]:
####TEST  MODEL

In [6]:
testdataset=pd.read_csv('/Users/swapnilgharat/Desktop/520/FINAL PROJECT/test.csv')
testdataset.shape

(1647, 67)

In [52]:
testdataset= testdataset.drop('Row', axis =1)

In [53]:
CAT=list(testdataset.select_dtypes(include=['object']))
CATdpX=testdataset.drop(CAT,axis =1).astype('float64')
CATdpX
list (CAT)

['x5', 'x13', 'x64', 'x65']

In [54]:
CATdpX.shape

(1647, 62)

In [55]:
#CREATE DUMMY COLUMNS FOR CATEGORICAL DATA
d5=pd.get_dummies(testdataset['x5'])
d6=pd.get_dummies(testdataset['x13'])
d7=pd.get_dummies(testdataset['x64'])
d8=pd.get_dummies(testdataset['x65'])

In [75]:
list(d8)

['NPT', 'PT', 'ST']

In [57]:
Xfortest=pd.concat([CATdpX,d5,d6,d7,d8],axis=1).astype('float')

In [58]:
import numpy as np
#Xnew.replace([np.inf, -np.inf], np.nan)
Xfortest.fillna(np.mean(Xtest), inplace = True)
Xfortest.shape

(1647, 73)

In [59]:
l= ['x5B','x5D','x64Mk','x64Mm','x65N']
for col in l:
    Xfortest[col]=0

In [60]:
Xfortest.shape

(1647, 78)

In [61]:
Xfortest.head()

,x1,x2,x3,x4,x6,x7,x8,x9,x10,x11,...,Ma,Mp,NPT,PT,ST,x5B,x5D,x64Mk,x64Mm,x65N
0,30.0,1.0,1.0,0.0,16.0,5.0,2.0,28.0,148.1,59.4,...,0.0,1.0,1.0,0.0,0.0,0,0,0,0,0
1,41.0,0.0,1.0,1.0,8.0,3.0,9.0,34.0,143.4,162.1,...,1.0,0.0,0.0,1.0,0.0,0,0,0,0,0
2,24.0,0.0,1.0,0.0,5.0,1.0,6.0,27.0,199.2,10.7,...,0.0,1.0,0.0,0.0,1.0,0,0,0,0,0
3,36.0,1.0,1.0,0.0,17.0,11.0,8.0,35.0,99.4,85.5,...,0.0,1.0,1.0,0.0,0.0,0,0,0,0,0
4,19.0,0.0,1.0,1.0,1.0,0.0,14.0,27.0,104.0,10.5,...,1.0,0.0,0.0,1.0,0.0,0,0,0,0,0


In [62]:
from sklearn.preprocessing import MinMaxScaler
scaling = MinMaxScaler()
XforStdtest = scaling.fit(Xfortest).transform(Xfortest)

In [63]:
yforpredtest = modsvm3.predict(XforStdtest)

In [64]:
yforpredtest

array([-1.,  1., -1., ...,  1., -1.,  1.])

In [76]:
print (yforpredtest)

[-1.  1. -1. ...,  1. -1.  1.]


# EXPORTING YPREDATEST VALUES TO EXCEL

In [77]:
import numpy as np
import pandas as pd
row_number=np.linspace(1,1647,1647)
yforpredtest_df = pd.DataFrame(row_number)

In [78]:
yforpredtest_df['yforpredtest'] = yforpredtest

In [79]:
yforpredtest_df.to_csv('/Users/swapnilgharat/Desktop/520/FINAL PROJECT/Classificationresults.csv',header=None,index=False)